In [61]:
!pip install -q numpy pandas scikit-learn matplotlib
!pip install -q sentence-transformers
!pip install -q umap-learn

In [48]:
import pandas as pd

DATA_PATH='data.csv'

df=pd.read_csv(DATA_PATH)
df.head()

,Text,Score,Suggestion
0,این اولین تجربه من برای خرید ایفون هست امروز...,100,1
1,خرید این محصول رو توصیه میکنم,84,1
2,1 ساله این گوشی رو دارم هیچ نقطه ضعفی ازش ند...,60,1
3,سلام خدمت دوستان این گوشی از همه نظر عالی کیف...,96,1
4,سلام دوستانی که نگران شکستن صفحه نمایش هستند ا...,92,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3261 entries, 0 to 3260
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Text        3261 non-null   object
 1   Score       3261 non-null   int64 
 2   Suggestion  3261 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 76.6+ KB


In [4]:
df['Suggestion'].value_counts().sort_index()

,count
Suggestion,
1,2382
2,419
3,460


In [49]:
def score_to_label(score):
    if score <40:
       return "negetive"
    elif score>=60:
       return "positive"
    else:
        return "neutral"
df['label']=df['Score'].apply(score_to_label)
df = df[df['label'] != 'neutral'].reset_index(drop=True)
df['label'].value_counts()

label
positive    2762
negetive     195
Name: count, dtype: int64

In [50]:
import re

# الگوی اموجی‌ها (ساده)
emoji_pattern = re.compile("""["
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # transport & map symbols
    u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
"]+""", flags=re.UNICODE)

def normalize_arabic_chars(text: str) -> str:
    # ي/ی و ك/ک
    text = text.replace('ي', 'ی').replace('ى', 'ی')
    text = text.replace('ك', 'ک')
    return text

def convert_persian_digits(text: str) -> str:
    persian_digits = '۰۱۲۳۴۵۶۷۸۹'
    english_digits = '0123456789'
    trans_table = str.maketrans(''.join(persian_digits), ''.join(english_digits))
    return text.translate(trans_table)

def clean_persian_text(text: str) -> str:
    if not isinstance(text, str):
        return ""

    text = normalize_arabic_chars(text)
    text = convert_persian_digits(text)

    # حذف URLها
    text = re.sub(r'http\S+|www\.\S+', ' ', text)

    # حذف ایمیل
    text = re.sub(r'\S+@\S+', ' ', text)

    # حذف @mention و هشتگ
    text = re.sub(r'[@#]\S+', ' ', text)

    # حذف اموجی‌ها
    text = emoji_pattern.sub(' ', text)

    # حذف هرچیزی به جز حروف و اعداد و فاصله
    text = re.sub(r'[^۰-۹0-9آ-یئءچژگۀ۱۲۳۴۵۶۷۸۹\s]', ' ', text)

    # تبدیل چند فاصله به یکی
    text = re.sub(r'\s+', ' ', text).strip()

    # کوچک‌سازی (در فارسی حساسیت کمتری دارد)
    text = text.lower()

    return text

df['clean_text'] = df['Text'].astype(str).apply(clean_persian_text)
df[['Text', 'clean_text']].head(10)

,Text,clean_text
0,این اولین تجربه من برای خرید ایفون هست امروز...,این اولین تجربه من برای خرید ایفون هست امروز ب...
1,خرید این محصول رو توصیه میکنم,خرید این محصول رو توصیه میکنم
2,1 ساله این گوشی رو دارم هیچ نقطه ضعفی ازش ند...,1 ساله این گوشی رو دارم هیچ نقطه ضعفی ازش ندیدم
3,سلام خدمت دوستان این گوشی از همه نظر عالی کیف...,سلام خدمت دوستان این گوشی از همه نظر عالی کیفی...
4,سلام دوستانی که نگران شکستن صفحه نمایش هستند ا...,سلام دوستانی که نگران شکستن صفحه نمایش هستند ا...
5,من پکیج کاملش روخریدم ازهرنظرعالیه هیچ نقد...,من پکیج کاملش روخریدم ازهرنظرعالیه هیچ نقدوایر...
6,اول از همه خوشحالم که بالاخره نسخه های 128 گیگ...,اول از همه خوشحالم که بالاخره نسخه های 128 گیگ...
7,من دو ماهه خریدم واقعا راضیم دوربینشم عالیه ...,من دو ماهه خریدم واقعا راضیم دوربینشم عالیه به...
8,بنظر من گوشی خوب و مقرون به صرفه ای بود توی ای...,بنظر من گوشی خوب و مقرون به صرفه ای بود توی ای...
9,از خریدم راضی ام و گوشی بینظیریه منتهی یه ایر...,از خریدم راضی ام و گوشی بینظیریه منتهی یه ایرا...


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt

tfidf_vectorixer= TfidfVectorizer(ngram_range=(1,2),
                                  min_df=5,
                                  max_df=0.9)

y=df['label']

X_train, X_test, y_train, y_test = train_test_split(df['clean_text'], y, test_size=0.2,random_state=42, stratify=y)

X_train = tfidf_vectorixer.fit_transform(X_train)
X_test = tfidf_vectorixer.transform(X_test)
X_train.shape


(2365, 4746)

In [8]:
type(X_train)

scipy.sparse._csr.csr_matrix

In [58]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

def accuracy(str,cl,X_test,y_test):
  y_pred = cl.predict(X_test)
  accuracy = accuracy_score(y_test, y_pred)

  # Precision
  precision = precision_score(y_test, y_pred, average='weighted')

  # Recall
  recall = recall_score(y_test, y_pred, average='weighted')

  # F1-score
  f1 = f1_score(y_test, y_pred, average='weighted')

  # Confusion matrix
  cm = confusion_matrix(y_test, y_pred)

  # گزارش کامل
  report = classification_report(y_test, y_pred)

  print("-" * (len(str) + 25))
  print(f"-------- {str} scores --------")
  print("-" * (len(str) + 25))
  print("Accuracy :", accuracy)
  print("Precision:", precision)
  print("Recall   :", recall)
  print("F1-score :", f1)
  print("\nConfusion Matrix:\n", cm)
  print("\nClassification Report:\n", report)


In [10]:
import warnings
from sklearn.exceptions import UndefinedMetricWarning
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

In [11]:
#TF-IDF
cl= LogisticRegression(max_iter=3000)
cl.fit(X_train,y_train)
accuracy("LogisticRegression",cl,X_test,y_test)

svm= LinearSVC()
svm.fit(X_train,y_train)
accuracy("LinearSVC",svm,X_test,y_test)

rf= RandomForestClassifier(n_estimators=300)
rf.fit(X_train,y_train)
accuracy("RandomForestClassifier",rf,X_test,y_test)

-------------------------------------------
-------- LogisticRegression scores --------
-------------------------------------------
Accuracy : 0.9341216216216216
Precision: 0.8725832039810081
Recall   : 0.9341216216216216
F1-score : 0.9023043786144224

Confusion Matrix:
 [[  0  39]
 [  0 553]]

Classification Report:
               precision    recall  f1-score   support

    negetive       0.00      0.00      0.00        39
    positive       0.93      1.00      0.97       553

    accuracy                           0.93       592
   macro avg       0.47      0.50      0.48       592
weighted avg       0.87      0.93      0.90       592

----------------------------------
-------- LinearSVC scores --------
----------------------------------
Accuracy : 0.9408783783783784
Precision: 0.9443975225225226
Recall   : 0.9408783783783784
F1-score : 0.9177240266375664

Confusion Matrix:
 [[  4  35]
 [  0 553]]

Classification Report:
               precision    recall  f1-score   support

    n

In [12]:
from sentence_transformers import SentenceTransformer
print("Loading BGE-M3 model...")
emb_model = SentenceTransformer("BAAI/bge-m3")
embeddings = emb_model.encode(
df['clean_text'].tolist(),
batch_size=32,
show_progress_bar=True
)
embeddings.shape


Loading BGE-M3 model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Batches:   0%|          | 0/93 [00:00<?, ?it/s]

(2957, 1024)

In [13]:
X_train_bge, X_test_bge, y_train_bge, y_test_bge = train_test_split(embeddings, y, test_size=0.2,random_state=42, stratify=y)


print(X_train_bge.shape)
print(X_test_bge.shape)

(2365, 1024)
(592, 1024)


In [14]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train_bge_enc = le.fit_transform(y_train_bge)
y_test_bge_enc  = le.transform(y_test_bge)

In [15]:
#BGE-M3
clbge= LogisticRegression(max_iter=3000)
clbge.fit(X_train_bge,y_train_bge)
accuracy("LogisticRegression", clbge, X_test_bge, y_test_bge)

svmbge= LinearSVC()
svmbge.fit(X_train_bge,y_train_bge)
accuracy("LinearSVC", svmbge, X_test_bge, y_test_bge)

rfbge= RandomForestClassifier(n_estimators=300)
rfbge.fit(X_train_bge, y_train_bge)
accuracy("RandomForestClassifier",rfbge, X_test_bge, y_test_bge)

-------------------------------------------
-------- LogisticRegression scores --------
-------------------------------------------
Accuracy : 0.9408783783783784
Precision: 0.9348220874458075
Recall   : 0.9408783783783784
F1-score : 0.9200569094116064

Confusion Matrix:
 [[  5  34]
 [  1 552]]

Classification Report:
               precision    recall  f1-score   support

    negetive       0.83      0.13      0.22        39
    positive       0.94      1.00      0.97       553

    accuracy                           0.94       592
   macro avg       0.89      0.56      0.60       592
weighted avg       0.93      0.94      0.92       592

----------------------------------
-------- LinearSVC scores --------
----------------------------------
Accuracy : 0.9425675675675675
Precision: 0.9312611460565424
Recall   : 0.9425675675675675
F1-score : 0.9318462649579671

Confusion Matrix:
 [[ 11  28]
 [  6 547]]

Classification Report:
               precision    recall  f1-score   support

    n

In [ ]:
from openai import OpenAI

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key="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",
)
batch_size = 100
embeddings_list = []

for i in range(0, len(df), batch_size):
    batch_texts = df['clean_text'].iloc[i:i+batch_size].tolist()
    resp = client.embeddings.create(
        model="openai/text-embedding-3-small",
        input=batch_texts
    )
    embeddings_list.extend([d.embedding for d in resp.data])

In [ ]:
print(embeddings_list[0])

In [ ]:
X_train_openai, X_test_openai, y_train_openai, y_test_openai = train_test_split(np.array(embeddings_list), y, test_size=0.2,random_state=42, stratify=y)


print(X_train_openai.shape)
print(X_test_openai.shape)

In [19]:
X_train_openai[0]

array([ 0.00593719,  0.02105174,  0.02232533, ...,  0.01273593,
       -0.03925662, -0.00127827])

In [20]:
#openAI
clopenai= LogisticRegression(max_iter=3000, class_weight='balanced')
clopenai.fit(X_train_openai,y_train_openai)
accuracy("LogisticRegression", clopenai, X_test_openai, y_test_openai)

svmopenai= LinearSVC()
svmopenai.fit(X_train_openai,y_train_openai)
accuracy("LinearSVC", svmopenai, X_test_openai, y_test_openai)

rfopenai= RandomForestClassifier(n_estimators=300)
rfopenai.fit(X_train_openai,y_train_openai)
accuracy("RandomForestClassifier",rfopenai, X_test_openai, y_test_openai)


-------------------------------------------
-------- LogisticRegression scores --------
-------------------------------------------
Accuracy : 0.7972972972972973
Precision: 0.9095179999803225
Recall   : 0.7972972972972973
F1-score : 0.8415196017935744

Confusion Matrix:
 [[ 21  18]
 [102 451]]

Classification Report:
               precision    recall  f1-score   support

    negetive       0.17      0.54      0.26        39
    positive       0.96      0.82      0.88       553

    accuracy                           0.80       592
   macro avg       0.57      0.68      0.57       592
weighted avg       0.91      0.80      0.84       592

----------------------------------
-------- LinearSVC scores --------
----------------------------------
Accuracy : 0.9341216216216216
Precision: 0.9082810489060489
Recall   : 0.9341216216216216
F1-score : 0.9083210582532881

Confusion Matrix:
 [[  2  37]
 [  2 551]]

Classification Report:
               precision    recall  f1-score   support

    n

In [21]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
import lightgbm  as lgb


In [22]:
#BGE-M3
knnbge = KNeighborsClassifier(n_neighbors=5)
knnbge.fit(X_train_bge,y_train_bge_enc)
accuracy("KNeighborsClassifier", knnbge, X_test_bge, y_test_bge_enc)


mlpbge  = MLPClassifier(hidden_layer_sizes=(128,64), max_iter=500, random_state=42)
mlpbge .fit(X_train_bge,y_train_bge_enc)
accuracy("MLPClassifier", mlpbge , X_test_bge, y_test_bge_enc)


xgbbge = XGBClassifier(n_estimators=300, learning_rate=0.05, use_label_encoder=False, eval_metric='logloss', random_state=42)
xgbbge.fit(X_train_bge,y_train_bge_enc)
accuracy("XGBClassifier", xgbbge, X_test_bge, y_test_bge_enc)


lgbmbge = lgb.LGBMClassifier(n_estimators=300, learning_rate=0.05, max_depth=7, random_state=42)
lgbmbge.fit(X_train_bge,y_train_bge_enc)
accuracy("LGBMClassifier", lgbmbge, X_test_bge, y_test_bge_enc)

---------------------------------------------
-------- KNeighborsClassifier scores --------
---------------------------------------------
Accuracy : 0.9358108108108109
Precision: 0.9215177908688517
Recall   : 0.9358108108108109
F1-score : 0.9253137730736187

Confusion Matrix:
 [[ 10  29]
 [  9 544]]

Classification Report:
               precision    recall  f1-score   support

           0       0.53      0.26      0.34        39
           1       0.95      0.98      0.97       553

    accuracy                           0.94       592
   macro avg       0.74      0.62      0.66       592
weighted avg       0.92      0.94      0.93       592

--------------------------------------
-------- MLPClassifier scores --------
--------------------------------------
Accuracy : 0.9358108108108109
Precision: 0.929826043915228
Recall   : 0.9358108108108109
F1-score : 0.9323733389711846

Confusion Matrix:
 [[ 16  23]
 [ 15 538]]

Classification Report:
               precision    recall  f1-score

/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [07:21:31] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


--------------------------------------
-------- XGBClassifier scores --------
--------------------------------------
Accuracy : 0.9425675675675675
Precision: 0.9321919126135993
Recall   : 0.9425675675675675
F1-score : 0.9271955288621956

Confusion Matrix:
 [[  8  31]
 [  3 550]]

Classification Report:
               precision    recall  f1-score   support

           0       0.73      0.21      0.32        39
           1       0.95      0.99      0.97       553

    accuracy                           0.94       592
   macro avg       0.84      0.60      0.65       592
weighted avg       0.93      0.94      0.93       592

[LightGBM] [Info] Number of positive: 2209, number of negative: 156
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.029185 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 261120
[LightGBM] [Info] Number of data points in the train set: 2365, number of used features: 1024
[LightGB

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [23]:
#OpenAI
le = LabelEncoder()
y_train_openai_enc = le.fit_transform(y_train_openai)
y_test_openai_enc  = le.transform(y_test_openai)


knnopenai = KNeighborsClassifier(n_neighbors=5)
knnopenai.fit(X_train_openai,y_train_openai_enc)
accuracy("KNeighborsClassifier", knnopenai, X_test_openai, y_test_openai_enc)


mlpopenai  = MLPClassifier(hidden_layer_sizes=(128,64), max_iter=500, random_state=42)
mlpopenai .fit(X_train_openai,y_train_openai_enc)
accuracy("MLPClassifier", mlpopenai , X_test_openai, y_test_openai_enc)


xgbopenai = XGBClassifier(n_estimators=300, learning_rate=0.05, use_label_encoder=False, eval_metric='logloss', random_state=42)
xgbopenai.fit(X_train_openai,y_train_openai_enc)
accuracy("XGBClassifier", xgbopenai, X_test_openai, y_test_openai_enc)


lgbmopenai = lgb.LGBMClassifier(n_estimators=300, learning_rate=0.05, max_depth=7, random_state=42)
lgbmopenai.fit(X_train_openai,y_train_openai_enc)
accuracy("LGBMClassifier", lgbmopenai, X_test_openai, y_test_openai_enc)

---------------------------------------------
-------- KNeighborsClassifier scores --------
---------------------------------------------
Accuracy : 0.9290540540540541
Precision: 0.8868260509231547
Recall   : 0.9290540540540541
F1-score : 0.9027010862537178

Confusion Matrix:
 [[  1  38]
 [  4 549]]

Classification Report:
               precision    recall  f1-score   support

           0       0.20      0.03      0.05        39
           1       0.94      0.99      0.96       553

    accuracy                           0.93       592
   macro avg       0.57      0.51      0.50       592
weighted avg       0.89      0.93      0.90       592

--------------------------------------
-------- MLPClassifier scores --------
--------------------------------------
Accuracy : 0.9037162162162162
Precision: 0.8895733214217255
Recall   : 0.9037162162162162
F1-score : 0.8962864155594549

Confusion Matrix:
 [[  5  34]
 [ 23 530]]

Classification Report:
               precision    recall  f1-scor

/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [07:22:41] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


--------------------------------------
-------- XGBClassifier scores --------
--------------------------------------
Accuracy : 0.9391891891891891
Precision: 0.9429059789840775
Recall   : 0.9391891891891891
F1-score : 0.9140859022645362

Confusion Matrix:
 [[  3  36]
 [  0 553]]

Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.08      0.14        39
           1       0.94      1.00      0.97       553

    accuracy                           0.94       592
   macro avg       0.97      0.54      0.56       592
weighted avg       0.94      0.94      0.91       592

[LightGBM] [Info] Number of positive: 2209, number of negative: 156
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.058139 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 391680
[LightGBM] [Info] Number of data points in the train set: 2365, number of used features: 1536
[LightGB

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [24]:
def accuracy_alltogether(names,models,X_test,y_test):

  for name, model in zip(names, models):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    # Precision
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)

    # Recall
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)

    # F1-score
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)

    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred)

    print(f"{name:30} {accuracy:<10.4f} {precision:<10.4f} {recall:<10.4f} {f1:<10.4f}")


In [25]:
warnings.filterwarnings("ignore", message="X does not have valid feature names")
print(f"{'Model':30} {'Accuracy':10} {'Precision':10} {'Recall':10} {'F1-score':10}")
print("-" * 80)
names1 = ["LogisticRegression-TFIDF", "LinearSVC-TFIDF", "RandomForestClassifier-TFIDF"]
models1 = [cl, svm, rf]

accuracy_alltogether(names1,models1,X_test,y_test)

print("-" * 80)

names2 = ["LogisticRegression-BGE", "LinearSVC-BGE", "RandomForestClassifier-BGE"]
models2 = [clbge, svmbge, rfbge]

accuracy_alltogether(names2,models2, X_test_bge, y_test_bge)

names2 = ["KNeighborsClassifier-BGE", "MLPClassifier-BGE", "XGBClassifier-BGE",  "LGBMClassifier-BGE"]
models2 = [ knnbge, mlpbge, xgbbge, lgbmbge]

accuracy_alltogether(names2,models2, X_test_bge, y_test_bge_enc)

print("-" * 80)
names3 = ["LogisticRegression-OpenAI", "LinearSVC-OpenAI", "RandomForestClassifier-OpenAI"]
models3 = [clopenai, svmopenai, rfopenai]

accuracy_alltogether(names3,models3, X_test_openai, y_test_openai)


names3 = ["KNeighborsClassifier-OpenAI", "MLPClassifier-OpenAI", "XGBClassifier-OpenAI",  "LGBMClassifier-OpenAI"]
models3 = [ knnopenai, mlpopenai, xgbopenai, lgbmopenai]

accuracy_alltogether(names3,models3, X_test_openai, y_test_openai_enc)

print("-" * 80)

Model                          Accuracy   Precision  Recall     F1-score  
--------------------------------------------------------------------------------
LogisticRegression-TFIDF       0.9341     0.8726     0.9341     0.9023    
LinearSVC-TFIDF                0.9409     0.9444     0.9409     0.9177    
RandomForestClassifier-TFIDF   0.9409     0.9348     0.9409     0.9201    
--------------------------------------------------------------------------------
LogisticRegression-BGE         0.9409     0.9348     0.9409     0.9201    
LinearSVC-BGE                  0.9426     0.9313     0.9426     0.9318    
RandomForestClassifier-BGE     0.9392     0.9429     0.9392     0.9141    
KNeighborsClassifier-BGE       0.9358     0.9215     0.9358     0.9253    
MLPClassifier-BGE              0.9358     0.9298     0.9358     0.9324    
XGBClassifier-BGE              0.9426     0.9322     0.9426     0.9272    
LGBMClassifier-BGE             0.9392     0.9252     0.9392     0.9210    
-------------

In [26]:
#BGE-M3
#with class_weight='balanced'
clbge2= LogisticRegression(max_iter=3000,class_weight='balanced')
clbge2.fit(X_train_bge,y_train_bge)
accuracy("LogisticRegression", clbge2, X_test_bge, y_test_bge)

svmbge2= LinearSVC(class_weight='balanced')
svmbge2.fit(X_train_bge,y_train_bge)
accuracy("LinearSVC", svmbge2, X_test_bge, y_test_bge)

rfbge2= RandomForestClassifier(n_estimators=300,class_weight='balanced')
rfbge2.fit(X_train_bge, y_train_bge)
accuracy("RandomForestClassifier",rfbge2, X_test_bge, y_test_bge)

ratio = len(y_train_bge_enc[y_train_bge_enc==0]) / len(y_train_bge_enc[y_train_bge_enc==1])

xgbbge2 = XGBClassifier(n_estimators=300, learning_rate=0.05, use_label_encoder=False, eval_metric='logloss', random_state=42,scale_pos_weight=ratio)
xgbbge2.fit(X_train_bge,y_train_bge_enc)
accuracy("XGBClassifier", xgbbge2, X_test_bge, y_test_bge_enc)


lgbmbge2 = lgb.LGBMClassifier(n_estimators=300, learning_rate=0.05, max_depth=7, random_state=42,class_weight='balanced')
lgbmbge2.fit(X_train_bge,y_train_bge_enc)
accuracy("LGBMClassifier", lgbmbge2, X_test_bge, y_test_bge_enc)

-------------------------------------------
-------- LogisticRegression scores --------
-------------------------------------------
Accuracy : 0.8783783783783784
Precision: 0.936470687967694
Recall   : 0.8783783783783784
F1-score : 0.8997025488486589

Confusion Matrix:
 [[ 29  10]
 [ 62 491]]

Classification Report:
               precision    recall  f1-score   support

    negetive       0.32      0.74      0.45        39
    positive       0.98      0.89      0.93       553

    accuracy                           0.88       592
   macro avg       0.65      0.82      0.69       592
weighted avg       0.94      0.88      0.90       592

----------------------------------
-------- LinearSVC scores --------
----------------------------------
Accuracy : 0.8952702702702703
Precision: 0.9354558915238527
Recall   : 0.8952702702702703
F1-score : 0.9105611083768774

Confusion Matrix:
 [[ 27  12]
 [ 50 503]]

Classification Report:
               precision    recall  f1-score   support

    ne

/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [07:24:37] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


--------------------------------------
-------- XGBClassifier scores --------
--------------------------------------
Accuracy : 0.9442567567567568
Precision: 0.9376818402749254
Recall   : 0.9442567567567568
F1-score : 0.9399552932186318

Confusion Matrix:
 [[ 17  22]
 [ 11 542]]

Classification Report:
               precision    recall  f1-score   support

           0       0.61      0.44      0.51        39
           1       0.96      0.98      0.97       553

    accuracy                           0.94       592
   macro avg       0.78      0.71      0.74       592
weighted avg       0.94      0.94      0.94       592

[LightGBM] [Info] Number of positive: 2209, number of negative: 156
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.031460 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 261120
[LightGBM] [Info] Number of data points in the train set: 2365, number of used features: 1024
[LightGB

In [27]:
#OpenAI
#with class_weight='balanced'
clopenai2= LogisticRegression(max_iter=3000,class_weight='balanced')
clopenai2.fit(X_train_openai,y_train_openai)
accuracy("LogisticRegression", clopenai2, X_test_openai, y_test_openai)

svmopenai2= LinearSVC(class_weight='balanced')
svmopenai2.fit(X_train_openai,y_train_openai)
accuracy("LinearSVC", svmopenai2, X_test_openai, y_test_openai)

rfopenai2= RandomForestClassifier(n_estimators=300,class_weight='balanced')
rfopenai2.fit(X_train_openai, y_train_openai)
accuracy("RandomForestClassifier",rfopenai2, X_test_openai, y_test_openai)

ratio = len(y_train_openai_enc[y_train_openai_enc==0]) / len(y_train_openai_enc[y_train_openai_enc==1])

xgbopenai2 = XGBClassifier(n_estimators=300, learning_rate=0.05, use_label_encoder=False, eval_metric='logloss', random_state=42,scale_pos_weight=ratio)
xgbopenai2.fit(X_train_openai,y_train_openai_enc)
accuracy("XGBClassifier", xgbopenai2, X_test_openai, y_test_openai_enc)


lgbmopenai2 = lgb.LGBMClassifier(n_estimators=300, learning_rate=0.05, max_depth=7, random_state=42,class_weight='balanced')
lgbmopenai2.fit(X_train_openai,y_train_openai_enc)
accuracy("LGBMClassifier", lgbmopenai2, X_test_openai, y_test_openai_enc)

-------------------------------------------
-------- LogisticRegression scores --------
-------------------------------------------
Accuracy : 0.7972972972972973
Precision: 0.9095179999803225
Recall   : 0.7972972972972973
F1-score : 0.8415196017935744

Confusion Matrix:
 [[ 21  18]
 [102 451]]

Classification Report:
               precision    recall  f1-score   support

    negetive       0.17      0.54      0.26        39
    positive       0.96      0.82      0.88       553

    accuracy                           0.80       592
   macro avg       0.57      0.68      0.57       592
weighted avg       0.91      0.80      0.84       592

----------------------------------
-------- LinearSVC scores --------
----------------------------------
Accuracy : 0.8361486486486487
Precision: 0.8920951224662161
Recall   : 0.8361486486486487
F1-score : 0.8612211957489505

Confusion Matrix:
 [[ 11  28]
 [ 69 484]]

Classification Report:
               precision    recall  f1-score   support

    n

/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [07:25:55] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


--------------------------------------
-------- XGBClassifier scores --------
--------------------------------------
Accuracy : 0.9358108108108109
Precision: 0.915878790878791
Recall   : 0.9358108108108109
F1-score : 0.9143866205167495

Confusion Matrix:
 [[  4  35]
 [  3 550]]

Classification Report:
               precision    recall  f1-score   support

           0       0.57      0.10      0.17        39
           1       0.94      0.99      0.97       553

    accuracy                           0.94       592
   macro avg       0.76      0.55      0.57       592
weighted avg       0.92      0.94      0.91       592

[LightGBM] [Info] Number of positive: 2209, number of negative: 156
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.043935 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 391680
[LightGBM] [Info] Number of data points in the train set: 2365, number of used features: 1536
[LightGBM

In [28]:
print("Using class_weight to Balanced the dataset")
warnings.filterwarnings("ignore", message="X does not have valid feature names")
print(f"{'Model':30} {'Accuracy':10} {'Precision':10} {'Recall':10} {'F1-score':10}")
print("-" * 80)

names2 = ["LogisticRegression-BGE", "LinearSVC-BGE", "RandomForestClassifier-BGE"]
models2 = [clbge2, svmbge2, rfbge2]

accuracy_alltogether(names2,models2, X_test_bge, y_test_bge)

names2 = [ "XGBClassifier-BGE",  "LGBMClassifier-BGE"]
models2 = [  xgbbge2, lgbmbge2]

accuracy_alltogether(names2,models2, X_test_bge, y_test_bge_enc)

print("-" * 80)
names3 = ["LogisticRegression-OpenAI", "LinearSVC-OpenAI", "RandomForestClassifier-OpenAI"]
models3 = [clopenai2, svmopenai2, rfopenai2]

accuracy_alltogether(names3,models3, X_test_openai, y_test_openai)


names3 = [ "XGBClassifier-OpenAI",  "LGBMClassifier-OpenAI"]
models3 = [  xgbopenai2, lgbmopenai2]

accuracy_alltogether(names3,models3, X_test_openai, y_test_openai_enc)

print("-" * 80)

Using class_weight to Balanced the dataset
Model                          Accuracy   Precision  Recall     F1-score  
--------------------------------------------------------------------------------
LogisticRegression-BGE         0.8784     0.9365     0.8784     0.8997    
LinearSVC-BGE                  0.8953     0.9355     0.8953     0.9106    
RandomForestClassifier-BGE     0.9409     0.9348     0.9409     0.9201    
XGBClassifier-BGE              0.9443     0.9377     0.9443     0.9400    
LGBMClassifier-BGE             0.9426     0.9311     0.9426     0.9304    
--------------------------------------------------------------------------------
LogisticRegression-OpenAI      0.7973     0.9095     0.7973     0.8415    
LinearSVC-OpenAI               0.8361     0.8921     0.8361     0.8612    
RandomForestClassifier-OpenAI  0.9392     0.9429     0.9392     0.9141    
XGBClassifier-OpenAI           0.9358     0.9159     0.9358     0.9144    
LGBMClassifier-OpenAI          0.9392     0.9

In [38]:
from imblearn.over_sampling import SMOTE

sm = SMOTE()
X_res, y_res = sm.fit_resample(X_train_bge, y_train_bge)

In [41]:
print(X_res.shape)
print(y_res.shape)
print(y_res.value_counts())

(4418, 1024)
(4418,)
label
positive    2209
negetive    2209
Name: count, dtype: int64


In [44]:
X_train_bge_sm, X_test_bge_sm, y_train_bge_sm, y_test_bge_sm = train_test_split(X_res, y_res, test_size=0.2,random_state=42, stratify=y_res)
le = LabelEncoder()
y_train_bge_sm_enc = le.fit_transform(y_train_bge_sm)
y_test_bge_sm_enc  = le.transform(y_test_bge_sm)

In [45]:
#BGE-M3
#Using OverSampling
clbge_sm= LogisticRegression(max_iter=3000)
clbge_sm.fit(X_train_bge_sm,y_train_bge_sm)
accuracy("LogisticRegression", clbge_sm, X_test_bge_sm, y_test_bge_sm)

svmbge_sm= LinearSVC()
svmbge_sm.fit(X_train_bge_sm,y_train_bge_sm)
accuracy("LinearSVC", svmbge_sm, X_test_bge_sm, y_test_bge_sm)

rfbge_sm= RandomForestClassifier(n_estimators=300)
rfbge_sm.fit(X_train_bge_sm, y_train_bge_sm)
accuracy("RandomForestClassifier",rfbge_sm, X_test_bge_sm, y_test_bge_sm)

knnbge_sm = KNeighborsClassifier(n_neighbors=5)
knnbge_sm.fit(X_train_bge_sm,y_train_bge_sm_enc)
accuracy("KNeighborsClassifier", knnbge_sm, X_test_bge_sm, y_test_bge_sm_enc)


mlpbge_sm  = MLPClassifier(hidden_layer_sizes=(128,64), max_iter=500, random_state=42)
mlpbge_sm .fit(X_train_bge_sm,y_train_bge_sm_enc)
accuracy("MLPClassifier", mlpbge_sm , X_test_bge_sm, y_test_bge_sm_enc)


xgbbge_sm = XGBClassifier(n_estimators=300, learning_rate=0.05, use_label_encoder=False, eval_metric='logloss', random_state=42)
xgbbge_sm.fit(X_train_bge_sm,y_train_bge_sm_enc)
accuracy("XGBClassifier", xgbbge_sm, X_test_bge_sm, y_test_bge_sm_enc)


lgbmbge_sm = lgb.LGBMClassifier(n_estimators=300, learning_rate=0.05, max_depth=7, random_state=42)
lgbmbge_sm.fit(X_train_bge_sm,y_train_bge_sm_enc)
accuracy("LGBMClassifier", lgbmbge_sm, X_test_bge_sm, y_test_bge_sm_enc)

-------------------------------------------
-------- LogisticRegression scores --------
-------------------------------------------
Accuracy : 0.8834841628959276
Precision: 0.8836432254727754
Recall   : 0.8834841628959276
F1-score : 0.8834720844664854

Confusion Matrix:
 [[395  47]
 [ 56 386]]

Classification Report:
               precision    recall  f1-score   support

    negetive       0.88      0.89      0.88       442
    positive       0.89      0.87      0.88       442

    accuracy                           0.88       884
   macro avg       0.88      0.88      0.88       884
weighted avg       0.88      0.88      0.88       884

----------------------------------
-------- LinearSVC scores --------
----------------------------------
Accuracy : 0.9502262443438914
Precision: 0.952906162464986
Recall   : 0.9502262443438914
F1-score : 0.9501525054466232

Confusion Matrix:
 [[437   5]
 [ 39 403]]

Classification Report:
               precision    recall  f1-score   support

    ne

/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [07:52:15] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


--------------------------------------
-------- XGBClassifier scores --------
--------------------------------------
Accuracy : 0.9830316742081447
Precision: 0.9835886214442012
Recall   : 0.9830316742081447
F1-score : 0.9830267872114651

Confusion Matrix:
 [[442   0]
 [ 15 427]]

Classification Report:
               precision    recall  f1-score   support

           0       0.97      1.00      0.98       442
           1       1.00      0.97      0.98       442

    accuracy                           0.98       884
   macro avg       0.98      0.98      0.98       884
weighted avg       0.98      0.98      0.98       884

[LightGBM] [Info] Number of positive: 1767, number of negative: 1767
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.045126 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 261120
[LightGBM] [Info] Number of data points in the train set: 3534, number of used features: 1024
[LightG

In [46]:
from imblearn.over_sampling import SMOTE

sm = SMOTE()
X_res, y_res = sm.fit_resample(X_train_openai, y_train_openai)

In [47]:
print(X_res.shape)
print(y_res.shape)
print(y_res.value_counts())

(4418, 1536)
(4418,)
label
positive    2209
negetive    2209
Name: count, dtype: int64


In [48]:
X_train_openai_sm, X_test_openai_sm, y_train_openai_sm, y_test_openai_sm = train_test_split(X_res, y_res, test_size=0.2,random_state=42, stratify=y_res)
le = LabelEncoder()
y_train_openai_sm_enc = le.fit_transform(y_train_openai_sm)
y_test_openai_sm_enc  = le.transform(y_test_openai_sm)

In [49]:
clopenai_sm= LogisticRegression(max_iter=3000)
clopenai_sm.fit(X_train_openai_sm,y_train_openai_sm)
accuracy("LogisticRegression", clopenai_sm, X_test_openai_sm, y_test_openai_sm)

svmopenai_sm= LinearSVC()
svmopenai_sm.fit(X_train_openai_sm,y_train_openai_sm)
accuracy("LinearSVC", svmopenai_sm, X_test_openai_sm, y_test_openai_sm)

rfopenai_sm= RandomForestClassifier(n_estimators=300)
rfopenai_sm.fit(X_train_openai_sm, y_train_openai_sm)
accuracy("RandomForestClassifier",rfopenai_sm, X_test_openai_sm, y_test_openai_sm)
knnopenai_sm = KNeighborsClassifier(n_neighbors=5)
knnopenai_sm.fit(X_train_openai_sm,y_train_openai_sm_enc)
accuracy("KNeighborsClassifier", knnopenai_sm, X_test_openai_sm, y_test_openai_sm_enc)


mlpopenai_sm  = MLPClassifier(hidden_layer_sizes=(128,64), max_iter=500, random_state=42)
mlpopenai_sm .fit(X_train_openai_sm,y_train_openai_sm_enc)
accuracy("MLPClassifier", mlpopenai_sm , X_test_openai_sm, y_test_openai_sm_enc)


xgbopenai_sm = XGBClassifier(n_estimators=300, learning_rate=0.05, use_label_encoder=False, eval_metric='logloss', random_state=42)
xgbopenai_sm.fit(X_train_openai_sm,y_train_openai_sm_enc)
accuracy("XGBClassifier", xgbopenai_sm, X_test_openai_sm, y_test_openai_sm_enc)


lgbmopenai_sm = lgb.LGBMClassifier(n_estimators=300, learning_rate=0.05, max_depth=7, random_state=42)
lgbmopenai_sm.fit(X_train_openai_sm,y_train_openai_sm_enc)
accuracy("LGBMClassifier", lgbmopenai_sm, X_test_openai_sm, y_test_openai_sm_enc)

-------------------------------------------
-------- LogisticRegression scores --------
-------------------------------------------
Accuracy : 0.8868778280542986
Precision: 0.8904028925619836
Recall   : 0.8868778280542986
F1-score : 0.8866218968515773

Confusion Matrix:
 [[413  29]
 [ 71 371]]

Classification Report:
               precision    recall  f1-score   support

    negetive       0.85      0.93      0.89       442
    positive       0.93      0.84      0.88       442

    accuracy                           0.89       884
   macro avg       0.89      0.89      0.89       884
weighted avg       0.89      0.89      0.89       884

----------------------------------
-------- LinearSVC scores --------
----------------------------------
Accuracy : 0.9547511312217195
Precision: 0.9581373762376237
Recall   : 0.9547511312217195
F1-score : 0.9546673640918344

Confusion Matrix:
 [[441   1]
 [ 39 403]]

Classification Report:
               precision    recall  f1-score   support

    n

/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [07:57:57] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


--------------------------------------
-------- XGBClassifier scores --------
--------------------------------------
Accuracy : 0.9909502262443439
Precision: 0.9910407110091742
Recall   : 0.9909502262443439
F1-score : 0.9909498093214917

Confusion Matrix:
 [[441   1]
 [  7 435]]

Classification Report:
               precision    recall  f1-score   support

           0       0.98      1.00      0.99       442
           1       1.00      0.98      0.99       442

    accuracy                           0.99       884
   macro avg       0.99      0.99      0.99       884
weighted avg       0.99      0.99      0.99       884

[LightGBM] [Info] Number of positive: 1767, number of negative: 1767
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.076508 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 391680
[LightGBM] [Info] Number of data points in the train set: 3534, number of used features: 1536
[LightG

In [50]:
print("Using Oversampling to Balanced the dataset")
warnings.filterwarnings("ignore", message="X does not have valid feature names")
print(f"{'Model':30} {'Accuracy':10} {'Precision':10} {'Recall':10} {'F1-score':10}")
print("-" * 80)

names2 = ["LogisticRegression-BGE", "LinearSVC-BGE", "RandomForestClassifier-BGE"]
models2 = [clbge_sm, svmbge_sm, rfbge_sm]

accuracy_alltogether(names2,models2, X_test_bge, y_test_bge)

names2 = [ "XGBClassifier-BGE",  "LGBMClassifier-BGE"]
models2 = [  xgbbge_sm, lgbmbge_sm]

accuracy_alltogether(names2,models2, X_test_bge, y_test_bge_enc)

print("-" * 80)
names3 = ["LogisticRegression-OpenAI", "LinearSVC-OpenAI", "RandomForestClassifier-OpenAI"]
models3 = [clopenai_sm, svmopenai_sm, rfopenai_sm]

accuracy_alltogether(names3,models3, X_test_openai, y_test_openai)


names3 = [ "XGBClassifier-OpenAI",  "LGBMClassifier-OpenAI"]
models3 = [  xgbopenai_sm, lgbmopenai_sm]

accuracy_alltogether(names3,models3, X_test_openai, y_test_openai_enc)

print("-" * 80)

Using Oversampling to Balanced the dataset
Model                          Accuracy   Precision  Recall     F1-score  
--------------------------------------------------------------------------------
LogisticRegression-BGE         0.8885     0.9382     0.8885     0.9068    
LinearSVC-BGE                  0.9003     0.9346     0.9003     0.9136    
RandomForestClassifier-BGE     0.9493     0.9433     0.9493     0.9450    
XGBClassifier-BGE              0.9459     0.9459     0.9459     0.9459    
LGBMClassifier-BGE             0.9510     0.9473     0.9510     0.9488    
--------------------------------------------------------------------------------
LogisticRegression-OpenAI      0.8243     0.9124     0.8243     0.8595    
LinearSVC-OpenAI               0.8699     0.8993     0.8699     0.8834    
RandomForestClassifier-OpenAI  0.9392     0.9429     0.9392     0.9141    
XGBClassifier-OpenAI           0.9274     0.8997     0.9274     0.9090    
LGBMClassifier-OpenAI          0.9358     0.9

فکر میکنم بهترین راه برای متوازن کردن دیتاست استفاده از دیتا آگمنتیشن روی کلاس منفی هستش
دیتا آگمنتیشن مثلا به صورت جایگزینی کلمات مترادف در جملات منفی که باعث اضافه شدن تعداد جملات منفی و متوازن شدن دیتاست میشه

یکی از این روش ها استفاده از llm ها برای ایجاد نمونه مشابه برای کلاس منفی است

In [2]:
!pip install openai

   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------- ----------------------------- 0.3/1.0 MB ? eta -:--:--
   ---------- ----------------------------- 0.3/1.0 MB ? eta -:--:--
   ---------- --------------------

In [ ]:
from openai import OpenAI
client = OpenAI(api_key)

def get_similar_words(word):
    completions = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "user",
                "content": f"{word}بدون هیچ توضیحاتی یه جمله با شباهت 50 درصد به این جمله بده: "
            }
        ],
        temperature=0
    )

    return completions.choices[0].message.content

In [21]:
neg_df=df[df["label"]  == 'negetive']
print(len(neg_df))
neg_df.head()

195


,Text,Score,Suggestion,label,clean_text
39,تو یه کلمه داغ میکنه زیاد دست رو میسوزونه عمل...,20,3,negetive,تو یه کلمه داغ میکنه زیاد دست رو میسوزونه عملا...
59,بسیار گوشی بیکیفت و ظریف و کند هست اصلا راضی ...,16,3,negetive,بسیار گوشی بیکیفت و ظریف و کند هست اصلا راضی ن...
85,لوازم جانبی نیست برای این مدل هر چند همون اول...,0,3,negetive,لوازم جانبی نیست برای این مدل هر چند همون اول ...
145,از لحاظ کیفیت صفحه کیبورد و تاج بدترین لپ تاب...,16,3,negetive,از لحاظ کیفیت صفحه کیبورد و تاج بدترین لپ تابی...
150,به درد نخور لپ تاپ دنیا,20,3,negetive,به درد نخور لپ تاپ دنیا


In [30]:
print(neg_df.iloc[0]["clean_text"])
word=get_similar_words( neg_df.iloc[0]["clean_text"])
print(word)

تو یه کلمه داغ میکنه زیاد دست رو میسوزونه عملا یه تومن پول رو دور ریختم با خریدم
یه وسیله خریدم که خیلی داغ میشه و دست رو میسوزونه، عملاً پولم رو به هدر دادم.


In [31]:
sent = neg_df["clean_text"].apply(get_similar_words)
print(sent)

39      یه وسیله خریدم که خیلی داغ میشه و دست رو میسوز...
59      گوشی بسیار بی‌کیفیت و شکننده است و عملکرد آن ب...
85      لوازم جانبی برای این مدل موجود نیست و حتی در ا...
145     از نظر کیفیت صفحه کلید و تاج، بدترین لپ‌تاپی ک...
150               لپ‌تاپ بی‌فایده‌ای که هیچ کارایی ندارد.
                              ...                        
2828    من یک تابه سفارش دادم و هنگام تحویل متوجه شدم ...
2837    متأسفانه بخارات غذا از لبه‌های ظرف خارج می‌شود...
2841    کیفیتش به اندازه یاماها نیست و باتری هم اگر بع...
2910    این پنکه را به خاطر سابقه خوب پارس خزر خریدم، ...
2950    سلام، وقت بخیر. لطفاً مدل‌های مناسب برای برش ف...
Name: clean_text, Length: 195, dtype: object


In [51]:
new_df = pd.DataFrame({
    "clean_text": sent,
    "label": "negetive"
})
new_df.head()

,clean_text,label
39,یه وسیله خریدم که خیلی داغ میشه و دست رو میسوز...,negetive
59,گوشی بسیار بی‌کیفیت و شکننده است و عملکرد آن ب...,negetive
85,لوازم جانبی برای این مدل موجود نیست و حتی در ا...,negetive
145,از نظر کیفیت صفحه کلید و تاج، بدترین لپ‌تاپی ک...,negetive
150,لپ‌تاپ بی‌فایده‌ای که هیچ کارایی ندارد.,negetive


In [52]:
df = pd.concat([df, new_df], ignore_index=True)


In [53]:
df['label'].value_counts()

label
positive    2762
negetive     390
Name: count, dtype: int64

به این شکل احتمالا بتوان دیتاست را به سمت توازن پیش برد و دقت را بالاتر برد

In [56]:
neg_df_count = len(df[df["label"]  == 'negetive'])
pos_df_count = len(df[df["label"]  == 'positive'])
need = int((pos_df_count - neg_df_count) / len(sent))
print(need)

12


In [57]:
prev_sent = sent
for i in range(need):
    new_sent = prev_sent.apply(get_similar_words)
    new_df = pd.DataFrame({
    "clean_text": new_sent,
    "label": "negetive"
    })
    df = pd.concat([df, new_df], ignore_index=True)
    prev_sent = new_sent

df['label'].value_counts()

label
positive    2762
negetive    2730
Name: count, dtype: int64